In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## 1단계: 구조화된 출력에 대한 Pydantic 모델 정의

이 모델들은 에이전트가 반환할 **스키마**를 정의합니다. Pydantic의 `response_format`을 사용하면 다음을 보장할 수 있습니다:
- ✅ 타입 안전한 데이터 추출
- ✅ 자동 검증
- ✅ 자유 텍스트 응답에서 발생하는 구문 분석 오류 없음
- ✅ 필드 기반의 간편한 조건부 라우팅


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## 2단계: 호텔 예약 도구 생성

이 도구는 **availability_agent**가 호출하여 객실 이용 가능 여부를 확인하는 데 사용됩니다. 우리는 `@ai_function` 데코레이터를 사용하여 다음을 수행합니다:
- Python 함수를 AI 호출 가능 도구로 변환
- LLM을 위한 JSON 스키마를 자동 생성
- 매개변수 유효성 검사 처리
- 에이전트에 의한 자동 호출 가능

이 데모에서는:
- **스톡홀름, 시애틀, 도쿄, 런던, 암스테르담** → 객실 있음 ✅
- **그 외 모든 도시** → 객실 없음 ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## 3단계: 라우팅을 위한 조건 함수 정의

이 함수들은 에이전트의 응답을 검사하여 워크플로에서 어떤 경로를 선택할지 결정합니다.

**핵심 패턴:**
1. 메시지가 `AgentExecutorResponse`인지 확인
2. 구조화된 출력(Pydantic 모델)을 분석
3. 라우팅을 제어하기 위해 `True` 또는 `False` 반환

워크플로는 **엣지**에서 이러한 조건을 평가하여 다음에 호출할 실행기를 결정합니다.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## 4단계: 사용자 지정 디스플레이 실행기 생성

실행기는 변환 또는 부수 효과를 수행하는 워크플로우 구성 요소입니다. 최종 결과를 표시하는 사용자 지정 실행기를 생성하기 위해 `@executor` 데코레이터를 사용합니다.

**핵심 개념:**
- `@executor(id="...")` - 함수를 워크플로우 실행기로 등록
- `WorkflowContext[Never, str]` - 입력/출력에 대한 타입 힌트
- `ctx.yield_output(...)` - 최종 워크플로우 결과를 반환


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## 5단계: 환경 변수 로드

LLM 클라이언트를 구성합니다. 이 예제는 다음과 함께 작동합니다:
- **GitHub Models** (GitHub 토큰을 사용하는 무료 등급)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## 6단계: 구조화된 출력으로 AI 에이전트 생성

우리는 **세 가지 전문화된 에이전트**를 생성하며, 각각 `AgentExecutor`로 감싸져 있습니다:

1. **availability_agent** - 도구를 사용하여 호텔 예약 가능 여부를 확인합니다.
2. **alternative_agent** - 객실이 없을 경우 대체 도시를 제안합니다.
3. **booking_agent** - 객실이 있을 경우 예약을 권장합니다.

**주요 기능:**
- `tools=[hotel_booking]` - 에이전트에 도구를 제공합니다.
- `response_format=PydanticModel` - 구조화된 JSON 출력을 강제합니다.
- `AgentExecutor(..., id="...")` - 워크플로우에서 사용할 수 있도록 에이전트를 감쌉니다.


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## 7단계: 조건부 엣지로 워크플로우 구축하기

이제 `WorkflowBuilder`를 사용하여 조건부 라우팅이 포함된 그래프를 구성합니다:

**워크플로우 구조:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**핵심 메서드:**
- `.set_start_executor(...)` - 시작 지점을 설정합니다
- `.add_edge(from, to, condition=...)` - 조건부 엣지를 추가합니다
- `.build()` - 워크플로우를 최종적으로 완성합니다


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## 8단계: 테스트 케이스 1 실행 - 이용 가능하지 않은 도시 (파리)

파리에서 호텔을 요청하여 **이용 가능하지 않은** 경로를 테스트해 봅시다 (우리의 시뮬레이션에서 방이 없는 도시).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## 9단계: 테스트 케이스 2 실행 - 이용 가능 도시 (스톡홀름)

이제 스톡홀름에서 호텔을 요청하여 **이용 가능** 경로를 테스트해 보겠습니다 (우리의 시뮬레이션에서 객실이 있는 도시).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## 주요 요점 및 다음 단계

### ✅ 배운 내용:

1. **WorkflowBuilder 패턴**
   - `.set_start_executor()`를 사용하여 시작 지점을 정의
   - `.add_edge(from, to, condition=...)`를 사용하여 조건부 라우팅 설정
   - `.build()`를 호출하여 워크플로를 최종적으로 완성

2. **조건부 라우팅**
   - 조건 함수는 `AgentExecutorResponse`를 검사
   - 구조화된 출력을 분석하여 라우팅 결정을 내림
   - `True`를 반환하면 엣지가 활성화되고, `False`를 반환하면 건너뜀

3. **도구 통합**
   - `@ai_function`을 사용하여 Python 함수를 AI 도구로 변환
   - 에이전트는 필요할 때 자동으로 도구를 호출
   - 도구는 에이전트가 분석할 수 있는 JSON을 반환

4. **구조화된 출력**
   - Pydantic 모델을 사용하여 타입 안전한 데이터 추출
   - 에이전트를 생성할 때 `response_format=MyModel`을 설정
   - 응답을 `Model.model_validate_json()`으로 분석

5. **사용자 정의 실행자**
   - `@executor(id="...")`를 사용하여 워크플로 구성 요소 생성
   - 실행자는 데이터를 변환하거나 부수 효과를 수행 가능
   - `ctx.yield_output()`을 사용하여 워크플로 결과 생성

### 🚀 실제 응용 사례:

- **여행 예약**: 가용성 확인, 대안 제안, 옵션 비교
- **고객 서비스**: 문제 유형, 감정, 우선순위에 따라 라우팅
- **전자상거래**: 재고 확인, 대안 제안, 주문 처리
- **콘텐츠 관리**: 독성 점수, 사용자 플래그에 따라 라우팅
- **승인 워크플로**: 금액, 사용자 역할, 위험 수준에 따라 라우팅
- **다단계 처리**: 데이터 품질, 완전성에 따라 라우팅

### 📚 다음 단계:

- 더 복잡한 조건 추가 (다중 기준)
- 워크플로 상태 관리를 통한 반복 구현
- 재사용 가능한 구성 요소를 위한 하위 워크플로 추가
- 실제 API(호텔 예약, 재고 시스템)와 통합
- 오류 처리 및 대체 경로 추가
- 내장된 시각화 도구를 사용하여 워크플로 시각화



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
